In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("create temp view") \
    .getOrCreate()

22/04/24 20:11:40 WARN Utils: Your hostname, diego-fliper resolves to a loopback address: 127.0.1.1; using 192.168.3.3 instead (on interface wlp3s0)
22/04/24 20:11:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/24 20:11:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/04/24 20:11:42 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/04/24 20:11:42 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/04/24 20:11:42 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [2]:
# imports and path
from pyspark.sql.types import DoubleType, StringType
from pyspark.sql import functions as func

path = "../datasets/alura/01/empresas/"

In [3]:
emp = spark.read.csv(path, sep=';', inferSchema=True)

In [4]:
empresasColNames = ['cnpj_basico', 'razao_social_nome_empresarial', 'natureza_juridica', 'qualificacao_do_responsavel', 'capital_social_da_empresa', 'porte_da_empresa', 'ente_federativo_responsavel']

In [5]:
for idx, name in enumerate(empresasColNames):
    emp = emp.withColumnRenamed(f"_c{idx}", name)

In [6]:
emp = emp.withColumn('capital_social_da_empresa', func.regexp_replace('capital_social_da_empresa', ',', '.'))
emp.limit(5).toPandas()

,cnpj_basico,razao_social_nome_empresarial,natureza_juridica,qualificacao_do_responsavel,capital_social_da_empresa,porte_da_empresa,ente_federativo_responsavel
0,306,FRANCAMAR REFRIGERACAO TECNICA S/C LTDA,2240,49,0.00,1,None
1,1355,BRASILEIRO & OLIVEIRA LTDA,2062,49,0.00,5,None
2,4820,"REGISTRO DE IMOVEIS, TABELIONATO 1 DE NOTAS E ...",3034,32,0.00,5,None
3,5347,ROSELY APARECIDA MONTEIRO CALTABIANO FREITAS,2135,50,0.00,5,None
4,6846,BADU E FILHOS TECIDOS LTDA,2062,49,4000.00,1,None


In [7]:
emp = emp.withColumn('capital_social_da_empresa', emp['capital_social_da_empresa'].cast(DoubleType()))
emp.limit(5).toPandas()

,cnpj_basico,razao_social_nome_empresarial,natureza_juridica,qualificacao_do_responsavel,capital_social_da_empresa,porte_da_empresa,ente_federativo_responsavel
0,306,FRANCAMAR REFRIGERACAO TECNICA S/C LTDA,2240,49,0.0,1,None
1,1355,BRASILEIRO & OLIVEIRA LTDA,2062,49,0.0,5,None
2,4820,"REGISTRO DE IMOVEIS, TABELIONATO 1 DE NOTAS E ...",3034,32,0.0,5,None
3,5347,ROSELY APARECIDA MONTEIRO CALTABIANO FREITAS,2135,50,0.0,5,None
4,6846,BADU E FILHOS TECIDOS LTDA,2062,49,4000.0,1,None


In [8]:
emp.where("capital_social_da_empresa==50").show(5, False)

+-----------+------------------------------------+-----------------+---------------------------+-------------------------+----------------+---------------------------+
|cnpj_basico|razao_social_nome_empresarial       |natureza_juridica|qualificacao_do_responsavel|capital_social_da_empresa|porte_da_empresa|ente_federativo_responsavel|
+-----------+------------------------------------+-----------------+---------------------------+-------------------------+----------------+---------------------------+
|17350147   |ERIK MARCELO DOS SANTOS 42107848858 |2135             |50                         |50.0                     |1               |null                       |
|17833214   |ALEXANDRE MACHADO LIMA 73750123772  |2135             |50                         |50.0                     |1               |null                       |
|20860830   |YASMIN MOURA DA FONSECA 13457709793 |2135             |50                         |50.0                     |1               |null                 

In [9]:
emp.select('cnpj_basico', 'porte_da_empresa', 'capital_social_da_empresa')\
    .groupBy("porte_da_empresa")\
    .agg(
        func.avg("capital_social_da_empresa").alias("capital_social_medio"),
        func.max("capital_social_da_empresa").alias("capital_social_medio_max"),
        func.count("cnpj_basico").alias("frequencia")
    )\
    .orderBy('porte_da_empresa', ascending=True)\
    .show()

+----------------+--------------------+------------------------+----------+
|porte_da_empresa|capital_social_medio|capital_social_medio_max|frequencia|
+----------------+--------------------+------------------------+----------+
|            null|    8.35421888053467|                 50000.0|      5985|
|               1|  339994.53313507047|        3.22014670262E11|   3129043|
|               3|  2601001.7677092687|        2.52006125741E11|    115151|
|               5|   708660.4208249793|                  5.0E10|   1335500|
+----------------+--------------------+------------------------+----------+



In [10]:
emp.select('capital_social_da_empresa')\
   .summary("mean", "stddev")\
   .show()

+-------+-------------------------+
|summary|capital_social_da_empresa|
+-------+-------------------------+
|   mean|        503694.5478542674|
| stddev|     2.1118691490537727E8|
+-------+-------------------------+



In [11]:
emp.createOrReplaceTempView("empresasView")

In [14]:
res = spark.sql("SELECT * FROM empresasView")
res.limit(5).toPandas()

,cnpj_basico,razao_social_nome_empresarial,natureza_juridica,qualificacao_do_responsavel,capital_social_da_empresa,porte_da_empresa,ente_federativo_responsavel
0,306,FRANCAMAR REFRIGERACAO TECNICA S/C LTDA,2240,49,0.0,1,None
1,1355,BRASILEIRO & OLIVEIRA LTDA,2062,49,0.0,5,None
2,4820,"REGISTRO DE IMOVEIS, TABELIONATO 1 DE NOTAS E ...",3034,32,0.0,5,None
3,5347,ROSELY APARECIDA MONTEIRO CALTABIANO FREITAS,2135,50,0.0,5,None
4,6846,BADU E FILHOS TECIDOS LTDA,2062,49,4000.0,1,None


In [15]:
res = spark.sql("""
                SELECT * FROM empresasView WHERE capital_social_da_empresa = 50
                """)
res.limit(5).toPandas()

,cnpj_basico,razao_social_nome_empresarial,natureza_juridica,qualificacao_do_responsavel,capital_social_da_empresa,porte_da_empresa,ente_federativo_responsavel
0,17350147,ERIK MARCELO DOS SANTOS 42107848858,2135,50,50.0,1,None
1,17833214,ALEXANDRE MACHADO LIMA 73750123772,2135,50,50.0,1,None
2,20860830,YASMIN MOURA DA FONSECA 13457709793,2135,50,50.0,1,None
3,22242856,JOAO CESAR MESSIAS 08707149883,2135,50,50.0,1,None
4,23238540,EVERTON ROBERTO DA SILVA 42101963809,2135,50,50.0,1,None


In [17]:
res = spark.sql("""
            SELECT porte_da_empresa, MEAN(capital_social_da_empresa) AS Media 
            FROM empresasView GROUP BY porte_da_empresa
                """)
res.limit(5).toPandas()

,porte_da_empresa,Media
0,NaN,8.354219e+00
1,1.0,3.399945e+05
2,3.0,2.601002e+06
3,5.0,7.086604e+05
